In [1]:
import sk8s
import sk8s.datastore

In [2]:
def datastore_service():
    import sk8s.datastore as ds
    datastore = ds.Datastore()
    datastore.run()

service = sk8s.services.service(datastore_service)
service

'http://service-bkuvu.sk8s.svc.cluster.local:5000'

In [4]:
import re, numpy, subprocess

service_name = re.search("://(.*?)\.", service).groups()[0]
remote_port = 5000
local_port = int(numpy.random.rand() * 10000 + 10000)
cmd = f"kubectl port-forward service/{service_name} {local_port}:{remote_port}"
print(cmd)

result = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(result)
local_url = f"http://localhost:{local_port}/"
local_url

kubectl port-forward service/service-bkuvu 10777:5000
<Popen: returncode: None args: 'kubectl port-forward service/service-bkuvu 1...>


'http://localhost:10777/'

In [5]:
sk8s.datastore.get(local_url)

{}

In [ ]:
import itertools

def run_with_logging(func, *args, name="job-{s}", results_url=None, **kwargs):

    if results_url is None:
        raise RuntimeError("results_url is None")

    s = sk8s.util.random_string(5)
    job = name.format(s=s)

    def logging_wrapper(func, *args, job_name=job, results_url=results_url):
        import sk8s.datastore
        import json
        result = func(*args)
        sk8s.datastore.put(results_url, job_name, result)
        return result

    return sk8s.run(lambda: logging_wrapper(func, *args), name=job, **kwargs)


def multiwait_with_logging(jobs, results_url, *args, **kwargs):
    results = [None]
    while None in results:
        status = sk8s.datastore.get(results_url)
        results = [status.get(job, default=None) for job in jobs]
    return results


def map_with_logging(func, iterable, results_url=None, **kwargs):
    jobs = list(map(lambda arg: run_with_logging(func, arg, results_url=results_url),
                    iterable))
    if kwargs.get("asynchro", False) == True:
        return jobs
    else:
        return multiwait_with_logging(jobs, results_url)
    


def func(n):
    import time
    time.sleep(5)
    return n * 2


jobs = map_with_logging(func, range(100), results_url=service, asynchro=True)

In [ ]:
results = sk8s.run(sk8s.datastore.get, service, asynchro=False)

In [ ]:
results